In [2]:
def limitedGetRequest(url, tagsToKeep=(), timeout=15,  maxConnectionAttempts=10):
    
    import ussl, usocket, gc

    scheme, _, host, target = url.split("/",3)

    if scheme == "https:":
        port = 443
    elif scheme == "http:":
        port = 80

    connectionAttempts = 0

    while connectionAttempts <= maxConnectionAttempts:
        connectionAttempts += 1

        try:
            ai = usocket.getaddrinfo(host, port, 0, usocket.SOCK_STREAM)[0]
            s = usocket.socket(ai[0], usocket.SOCK_STREAM, ai[2])
            s.settimeout(timeout) # time for connection to timeout in seconds
        except:
            print("Issue setting up socket")
            s.close()
            continue 

        try:
            s.connect(ai[-1])
            s = ussl.wrap_socket(s, server_hostname=host)
            s.setblocking(True)
        except:
            print("Issue connecting to server or wrapping socket")
            s.close()
            continue
    
        try:
            s.write(f"GET /{target} HTTP/1.0\r\nHost: {host}\r\nUser-agent: Mozilla/5.0\r\nConnection: close\r\n\r\n")
        except:
            print("Issue writing to socket")
            s.close()
            continue
    
        try:
            keptLines = []
            strBuff = []
            buildString = False
            keepLine = False
            while True:
                
                buff = s.read(512)
                gc.collect()
                
                if buff != b"":
                    for x in buff:
                        
                        x = chr(x)

                        if buildString:
                            strBuff.append(x)
                        
                        if x == "<" and not buildString:
                            buildString = True
                            strBuff.append(x)
                        
                        elif x == ">" and buildString and not keepLine:
                            bufferTagName = "".join(strBuff[1:-1]).split(" ",1)[0]
                            print(bufferTagName)
                            if any( tag in bufferTagName for tag in tagsToKeep ) and strBuff[1] != "/":
                                keepLine = True
                            else:
                                buildString = False
                                keepLine = False
                                strBuff = []
                            
                        elif x == ">" and buildString and keepLine:
                            closingTag = "</" + bufferTagName + ">"
                            if "".join(strBuff[-len(closingTag):]) == closingTag:
                                keptLines.append( "".join(strBuff) )
                                buildString = False
                                keepLine = False
                                strBuff = []
                            
                else:
                    break

            s.close()
            del s
            gc.collect()
            return (keptLines)

        except:
            print("Issue reading from socket")
            s.close()
            pass

    raise

import parameters as params
tagsToKeep = ["start-valid-time", "temperature", "probability-of-precipitation", "cloud-amount"]
xmlWeatherResponseLines = limitedGetRequest(f"https://forecast.weather.gov/MapClick.php?lat={params.lat}&lon={params.long}&FcstType=digitalDWML", tagsToKeep, timeout=60)
